### Student Information
Name: 黃宇加

Student ID: 110034011

GitHub ID: REX722

Kaggle name: IEEM-ML2023-110034011

Kaggle private scoreboard snapshot:

![Snapshot](../pics/kaggle.png)

# 載入 package

In [17]:
import json
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Data preparation (將資料轉換成DataFrame的格式並輸出成 pickle 檔)

In [ ]:
data = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
f.close()

In [ ]:
emotion = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')

In [ ]:
df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})
df = df.merge(data_identification, on='tweet_id', how='left')

train_df = df[df['identification'] == 'train']
test_df = df[df['identification'] == 'test']

In [ ]:
train_df = train_df.merge(emotion, on='tweet_id', how='left')

In [ ]:
train_df.to_pickle("/kaggle/working/train_df.pkl") 
test_df.to_pickle("/kaggle/working/test_df.pkl")

# Data preprocessing

## 載入先前處理完的 DataFrame

In [ ]:
train_df = pd.read_pickle("/kaggle/input/dm2024-lab2-competition-dataset/train_df.pkl")

In [57]:
train_df

,tweet_id,hashtags,text,identification,emotion
0,0x376b20,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,0x2d5350,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,0x1cd5b0,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,0x1d755c,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,0x2c91a8,[],Still waiting on those supplies Liscus. <LH>,train,anticipation
...,...,...,...,...,...
1455558,0x321566,"[NoWonder, Happy]",I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy
1455559,0x38959e,[],In every circumtance I'd like to be thankful t...,train,joy
1455560,0x2cbca6,[blessyou],there's currently two girls walking around the...,train,joy
1455561,0x24faed,[],"Ah, corporate life, where you can date <LH> us...",train,joy


## 刪除 duplicate 的 text

In [59]:
train_df.drop_duplicates(subset=['text'], keep=False, inplace=True)

In [60]:
train_df

,tweet_id,hashtags,text,identification,emotion
0,0x376b20,[Snapchat],"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,0x2d5350,"[freepress, TrumpLegacy, CNN]","@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,0x1cd5b0,[],Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,0x1d755c,"[authentic, LaughOutLoud]",@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,0x2c91a8,[],Still waiting on those supplies Liscus. <LH>,train,anticipation
...,...,...,...,...,...
1455558,0x321566,"[NoWonder, Happy]",I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy
1455559,0x38959e,[],In every circumtance I'd like to be thankful t...,train,joy
1455560,0x2cbca6,[blessyou],there's currently two girls walking around the...,train,joy
1455561,0x24faed,[],"Ah, corporate life, where you can date <LH> us...",train,joy


# Model

In [61]:
data = train_df.drop(['tweet_id','identification', 'hashtags'], axis=1)

In [74]:
data

,text,emotion
0,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,Still waiting on those supplies Liscus. <LH>,anticipation
...,...,...
1455558,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy
1455559,In every circumtance I'd like to be thankful t...,joy
1455560,there's currently two girls walking around the...,joy
1455561,"Ah, corporate life, where you can date <LH> us...",joy


In [63]:
pip install gensim scikit-learn pandas

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 46.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tsfresh 0.20.3 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
Note: you may need to restart the kernel to use 

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import lightgbm as lgb

texts = data["text"].apply(lambda x: x.split())  # 將每行文本切分為單詞列表
emotions = data["emotion"]

label_encoder = LabelEncoder()
emotions_encoded = label_encoder.fit_transform(emotions)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, emotions_encoded, test_size=0.2, random_state=42
)

## 訓練 Word2Vec 的模型

In [ ]:
word2vec_model = Word2Vec(
    sentences=train_texts,
    vector_size=200,
    window=10,
    min_count=5,
    workers=8,
    epochs=10,
    sg = 1
)

# 利用平均的方式表現一個句字
def get_sentence_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# 將文本轉換為向量
train_vectors = np.array([get_sentence_vector(sentence, word2vec_model) for sentence in train_texts])
test_vectors = np.array([get_sentence_vector(sentence, word2vec_model) for sentence in test_texts])

## 使用 LightGBM 做分類任務

In [126]:
# LightGBM 設置參數
params = {
    "objective": "multiclass",
    "num_class": len(set(emotions_encoded)),
    "boosting_type": "gbdt",
    "metric": "multi_logloss",
    "learning_rate": 0.1,
    "max_depth": 7,
    "num_leaves": 127, 
    "min_data_in_leaf": 20,
    "n_estimators": 100,
    "device_type": "gpu",
    "n_jobs": -1
}

# 構建 LightGBM 數據集
train_dataset = lgb.Dataset(train_vectors, label=train_labels)
test_dataset = lgb.Dataset(test_vectors, label=test_labels, reference=train_dataset)

# 訓練模型
model = lgb.train(
    params,
    train_dataset,
    valid_sets=[train_dataset, test_dataset],
    callbacks=[
        lgb.early_stopping(stopping_rounds=3),
        lgb.log_evaluation(period=10)
    ]
)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 1159345, number of used features: 200
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 200 dense feature groups (221.13 MB) transferred to GPU in 0.170633 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -3.604290
[LightGBM] [Info] Start training from score -1.761783
[LightGBM] [Info] Start training from score -2.343480
[LightGBM] [Info] Start training from score -3.123961
[LightGBM] [Info] Start training from score -1.039014
[LightGBM] [Info] Start training from score -2.014756
[LightGBM] [Info] Start training from score -3.413299
[LightGBM] [Info] Start training from score -1.959061
[LightGBM] [Warning] No further splits with posit

In [127]:
# 預測
predictions = model.predict(test_vectors)
predicted_classes = np.argmax(predictions, axis=1)

# 評估結果
print("Accuracy:", accuracy_score(test_labels, predicted_classes))
print("Classification Report:\n", classification_report(test_labels, predicted_classes, target_names=label_encoder.classes_))

Accuracy: 0.5135783216083523
Classification Report:
               precision    recall  f1-score   support

       anger       0.63      0.12      0.21      8037
anticipation       0.62      0.49      0.54     49420
     disgust       0.38      0.34      0.36     27637
        fear       0.65      0.25      0.36     12711
         joy       0.51      0.84      0.63    103001
     sadness       0.44      0.42      0.43     38446
    surprise       0.85      0.09      0.16      9529
       trust       0.61      0.20      0.30     41056

    accuracy                           0.51    289837
   macro avg       0.58      0.34      0.37    289837
weighted avg       0.54      0.51      0.48    289837



---

# 預測 test_df 中的資料

In [131]:
test_df = pd.read_pickle("/kaggle/input/dm2024-lab2-competition-dataset/test_df.pkl")
test_df = test_df.drop(['identification', 'hashtags'], axis=1)

In [132]:
test_df

,tweet_id,text
2,0x28b412,"Confident of your obedience, I write to you, k..."
4,0x2de201,"""Trust is not the same as faith. A friend is s..."
9,0x218443,When do you have enough ? When are you satisfi...
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #..."
33,0x26289a,"In these tough times, who do YOU turn to as yo..."
...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th..."
1867529,0x2a980e,"""There is a lad here, which hath five barley l..."
1867530,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,0x29d0cb,I swear all this hard work gone pay off one da...


In [133]:
# 預處理新數據：將文本切分為單詞列表
test_df["text_split"] = test_df["text"].apply(lambda x: x.split())

In [134]:
# 計算文本向量的平均值
test_vectors_new = np.array([get_sentence_vector(sentence, word2vec_model) for sentence in test_df["text_split"]])

In [ ]:
# 使用訓練好的模型進行預測
predictions_new = model.predict(test_vectors_new)

predicted_classes_new = np.argmax(predictions_new, axis=1)
predicted_emotions = label_encoder.inverse_transform(predicted_classes_new)

test_df["predicted_emotion"] = predicted_emotions

In [136]:
test_df

,tweet_id,text,text_split,predicted_emotion
2,0x28b412,"Confident of your obedience, I write to you, k...","[Confident, of, your, obedience,, I, write, to...",anticipation
4,0x2de201,"""Trust is not the same as faith. A friend is s...","[""Trust, is, not, the, same, as, faith., A, fr...",anticipation
9,0x218443,When do you have enough ? When are you satisfi...,"[When, do, you, have, enough, ?, When, are, yo...",sadness
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #...","[God, woke, you, up,, now, chase, the, day, #G...",joy
33,0x26289a,"In these tough times, who do YOU turn to as yo...","[In, these, tough, times,, who, do, YOU, turn,...",anticipation
...,...,...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th...","[""For, this, is, the, message, that, ye, heard...",anticipation
1867529,0x2a980e,"""There is a lad here, which hath five barley l...","[""There, is, a, lad, here,, which, hath, five,...",anticipation
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,"[When, you, buy, the, last, 2, tickets, remain...",joy
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,"[I, swear, all, this, hard, work, gone, pay, o...",joy


---

# 產生最後交出去的檔案

In [137]:
submission = test_df[['tweet_id', 'predicted_emotion']]

In [138]:
submission = submission.rename(columns = {'tweet_id': 'id', 'predicted_emotion': 'emotion'})

In [139]:
submission.to_csv('/kaggle/working/submission.csv', index = False)

---

# 結果與討論

1. 一開始我使用 randomforest 配合 Tf-IDF 做為 baseline，Public 的準確度大概在0.31左右。
2. 後來利用 Word2Vec 的值當做 feature，並用 LightGBM 做分類任務，因為數據集較大，用 LightGBM 比較快，Public 的準確度大概在 0.34 左右。
3. 調整 Word2Vec 的參數，像是 vector_size, min_count, epochs 等等，雖然訓練時間拉長了一些，但是 Public 的準確度達到了 0.407。
4. 後來嘗試使用 Bert，不過因為資料太大了，需要訓練的時間太長，所以沒能成功。